In [1]:
import pandas as pd

from tadv.utils import get_project_root

result_table_path = get_project_root() / 'workflow' / 's3_evaluation' / 'result_tables' / 'playground-series-s4e10__ml_inference_classification__0.csv'

df = pd.read_csv(result_table_path)

In [2]:
df

,Unnamed: 0,Script,Execution Result on Clean Data,Execution Result on Corrupted Data,Model,Strategy,Passed Constraints (Clean),Passed Constraints (Corrupted),Failed Constraints (Clean),Failed Constraints (Corrupted)
0,0,classification_0,0.695405,error,deequ,NaN,30,28,4,6
1,1,classification_0,0.695405,error,gpt-4o,NaN,5,5,0,0
2,2,classification_0,0.695405,error,gpt-4o,with_deequ,12,11,0,1
3,3,classification_0,0.695405,error,gpt-4o,with_experience,13,11,0,2
4,4,classification_1,0.830201,error,deequ,NaN,30,28,4,6
5,5,classification_1,0.830201,error,gpt-4o,NaN,10,9,0,1
6,6,classification_1,0.830201,error,gpt-4o,with_deequ,25,23,0,2
7,7,classification_1,0.830201,error,gpt-4o,with_experience,24,22,0,2
8,8,classification_2,0.500000,0.5,deequ,NaN,30,28,4,6
9,9,classification_2,0.500000,0.5,gpt-4o,NaN,10,9,0,1


In [8]:
def performance_drop(result_on_clean_data, result_on_corrupted_data):
    
    if result_on_corrupted_data == "error":
        return True
    else:
        result_on_clean_data = float(result_on_clean_data)
        result_on_corrupted_data = float(result_on_corrupted_data)
        if (result_on_clean_data - result_on_corrupted_data) / result_on_clean_data > 0.05:
            return True
        else:
            return False
# df['performance_drop'] = df.apply(lambda x: performance_drop(x['Execution Result on Clean Data'], x['Execution Result on Corrupted Data']), axis=1)
df['performance_drop'] = df.apply(lambda x: performance_drop(x['Execution Result on Clean Data'], x['Execution Result on Corrupted Data']), axis=1)

df['passed_constraints_ratio_on_clean_data'] = df['Passed Constraints (Clean)'] / (df['Passed Constraints (Clean)'] + df['Failed Constraints (Clean)'])
df['passed_constraints_ratio_on_corrupted_data'] = df['Passed Constraints (Corrupted)'] / (df['Passed Constraints (Corrupted)'] + df['Failed Constraints (Corrupted)'])

df.columns

Index(['Unnamed: 0', 'Script', 'Execution Result on Clean Data',
       'Execution Result on Corrupted Data', 'Model', 'Strategy',
       'Passed Constraints (Clean)', 'Passed Constraints (Corrupted)',
       'Failed Constraints (Clean)', 'Failed Constraints (Corrupted)',
       'performance_drop', 'passed_constraints_ratio_on_clean_data',
       'passed_constraints_ratio_on_corrupted_data', 'passed_on_clean_data',
       'passed_on_corrupted_data'],
      dtype='object')

In [12]:
df['Strategy'].fillna("None", inplace=True)
unique_combinations = df[['Model', 'Strategy']].drop_duplicates()
for model, strategy in unique_combinations.values:
    print(model, strategy)
    df_ = df[(df['Model'] == model) & (df['Strategy'] == strategy)]
    clean_no_impact_fail = (df_['Failed Constraints (Clean)'] > 0).sum()
    clean_no_impact_pass = (df_['Failed Constraints (Clean)'] == 0).sum()
    df_with_no_impact_errors = df_[df_['performance_drop'] == False]
    error_no_impact_fail = (df_with_no_impact_errors['Failed Constraints (Corrupted)'] > 0).sum()
    error_no_impact_pass = (df_with_no_impact_errors['Failed Constraints (Corrupted)'] == 0).sum()
    df_with_impact_errors = df_[df_['performance_drop'] == True]
    error_impact_fail = (df_with_impact_errors['Failed Constraints (Corrupted)'] > 0).sum()
    error_impact_pass = (df_with_impact_errors['Failed Constraints (Corrupted)'] == 0).sum()
    print(clean_no_impact_fail, clean_no_impact_pass)
    print(error_no_impact_fail, error_no_impact_pass)
    print(error_impact_fail, error_impact_pass)

    

deequ None
15 0
7 0
8 0
gpt-4o None
4 11
4 3
6 2
gpt-4o with_deequ
4 11
7 0
8 0
gpt-4o with_experience
3 12
7 0
8 0


/var/folders/19/8dpm1qss7mn34db1f49thmww0000gn/T/ipykernel_2166/2069450294.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Strategy'].fillna("None", inplace=True)


In [15]:
results = []

for model, strategy in unique_combinations.values:
    df_ = df[(df['Model'] == model) & (df['Strategy'] == strategy)]
    clean_no_impact_fail = (df_['Failed Constraints (Clean)'] > 0).sum()
    clean_no_impact_pass = (df_['Failed Constraints (Clean)'] == 0).sum()
    df_with_no_impact_errors = df_[df_['performance_drop'] == False]
    error_no_impact_fail = (df_with_no_impact_errors['Failed Constraints (Corrupted)'] > 0).sum()
    error_no_impact_pass = (df_with_no_impact_errors['Failed Constraints (Corrupted)'] == 0).sum()
    df_with_impact_errors = df_[df_['performance_drop'] == True]
    error_impact_fail = (df_with_impact_errors['Failed Constraints (Corrupted)'] > 0).sum()
    error_impact_pass = (df_with_impact_errors['Failed Constraints (Corrupted)'] == 0).sum()

    results.append([model, strategy, clean_no_impact_fail, clean_no_impact_pass,
                    error_no_impact_fail, error_no_impact_pass,
                    error_impact_fail, error_impact_pass])

result_df = pd.DataFrame(results, columns=[
    'Model', 'Strategy', 
    'Clean Fail', 'Clean Pass', 
    'Error NoImpact Fail', 'Error NoImpact Pass', 
    'Error Impact Fail', 'Error Impact Pass'
])

result_df

,Model,Strategy,Clean Fail,Clean Pass,Error NoImpact Fail,Error NoImpact Pass,Error Impact Fail,Error Impact Pass
0,deequ,None,15,0,7,0,8,0
1,gpt-4o,None,4,11,4,3,6,2
2,gpt-4o,with_deequ,4,11,7,0,8,0
3,gpt-4o,with_experience,3,12,7,0,8,0


In [10]:
df['Strategy'].fillna("None", inplace=True)
unique_combinations = df[['Model', 'Strategy']].drop_duplicates()
for model, strategy in unique_combinations.values:
    print(model, strategy)
    df_ = df[(df['Model'] == model) & (df['Strategy'] == strategy)]
    print((df_['Failed Constraints (Clean)'] == 0).sum(), (df_['Failed Constraints (Clean)'] > 0).sum())
    print((df_['Failed Constraints (Corrupted)'] == 0).sum(), (df_['Failed Constraints (Corrupted)'] > 0).sum())

deequ None
0 15
0 15
gpt-4o None
11 4
5 10
gpt-4o with_deequ
11 4
0 15
gpt-4o with_experience
12 3
0 15


/var/folders/19/8dpm1qss7mn34db1f49thmww0000gn/T/ipykernel_2166/1497558433.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Strategy'].fillna("None", inplace=True)


In [14]:
results = []

for model, strategy in unique_combinations.values:
    df_ = df[(df['Model'] == model) & (df['Strategy'] == strategy)]
    
    clean_pass = (df_['Failed Constraints (Clean)'] == 0).sum()
    clean_fail = (df_['Failed Constraints (Clean)'] > 0).sum()
    corrupted_pass = (df_['Failed Constraints (Corrupted)'] == 0).sum()
    corrupted_fail = (df_['Failed Constraints (Corrupted)'] > 0).sum()
    
    results.append([model, strategy, clean_pass, clean_fail, corrupted_pass, corrupted_fail])

# Convert to DataFrame
result_df = pd.DataFrame(results, columns=['Model', 'Strategy', 'Clean Pass', 'Clean Fail', 'Corrupted Pass', 'Corrupted Fail'])
result_df

,Model,Strategy,Clean Pass,Clean Fail,Corrupted Pass,Corrupted Fail
0,deequ,None,0,15,0,15
1,gpt-4o,None,11,4,5,10
2,gpt-4o,with_deequ,11,4,0,15
3,gpt-4o,with_experience,12,3,0,15
